In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
import dask

import datacube 
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.utils import geometry

import fiona
import rasterio.features
from osgeo import gdal, ogr
import os
from rsgislib.segmentation import segutils
from rasterstats import zonal_stats

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools, BandIndices
from load_data import load_data
from transform_tuple import transform_tuple
from imageSeg import imageSeg
from query_from_shp import query_from_shp


# %load_ext autoreload
# %autoreload 2

Set up query and analysis parameters
Here we set a centroid for the area we want to compare, and set up CRS, resolution and resampling that will be applied to both collectiondatasets. The values below extract both collections to match the collection upgrade CRS and resolution (UTM zone 56 S and 30m pixels).

In [ ]:
# where is your data and results folder?
data = 'data/'
results = 'results/'

#Input your area of interest's name, coords, and 
#the year you're interested in?
AOI = 'collectionTest'
year = 'Summer2017-18'
# Centre point of spatial query
lat, lon = -35.969, 145.394
time_period = ('2017-11-01', '2018-03-31')

# Desired output resolution and projection for both datasets
output_crs = 'EPSG:28355'
output_resolution = (30, 30)
output_resamp_continuous = 'bilinear'
output_resamp_categorical = 'nearest'

# Bands/measurements to load
collectionupgrade_bands = ['nbart_red','nbart_nir']
currentcollection_bands = ['red', 'nir']

# Set up query
query = {'lon': (lon - 0.15, lon + 0.15),
         'lat': (lat - 0.15, lat + 0.15),
         'time': time_period}

# Connect to current collection and collection upgrade databases
dc_ard = datacube.Datacube(config='/g/data1a/u46/users/cb3058/datacube.conf', env='datacube_upgrade')
dc = datacube.Datacube()

# Preview spatial query
# DEAPlotting.display_map(x=query['lon'], y=query['lat'], crs='EPSG:4326')

#What thresholds should I use?
threshold = 0.8

In [ ]:
#Creating a folder to keep things neat
directory = results + AOI + "_" + year
if not os.path.exists(directory):
    os.mkdir(directory)

results = results + AOI + "_" + year + "/"


## Load in collection upgrade data (DEA Landsat Collection 3)

In [ ]:
xarray_dict = {}

# Iterate through each sensor
for sensor in ['ls8']:

    # Load data 
    landsat_ds = dc_ard.load(product = f'{sensor}_ard', 
                             measurements = collectionupgrade_bands,
                             output_crs = output_crs,
                             resolution = output_resolution,
                             resampling = output_resamp_continuous,
                             align=(output_resolution[0]/2.0, output_resolution[1]/2.0),
                             group_by = 'solar_day', 
                             **query)
    
    # Load PQ data seperately (this enables using a different resampling method on 
    # continuous surface reflectance values vs categorical fmask/PQ values)
    landsat_pq = dc_ard.load(product = f'{sensor}_ard', 
                             measurements = ['fmask'],
                             output_crs = output_crs,
                             resolution = output_resolution,
                             resampling = output_resamp_categorical,
                             align=(output_resolution[0]/2.0, output_resolution[1]/2.0),
                             group_by = 'solar_day', 
                             **query)

    # Identify pixels with valid data: no nodata AND no cloud AND no cloud shadow
    good_quality = ((landsat_pq.fmask != 0) & 
                    (landsat_pq.fmask != 2) & 
                    (landsat_pq.fmask != 3))

    # Apply mask to set all PQ-affected pixels to NaN and set nodata to NaN
    landsat_ds = landsat_ds.where(good_quality)

    # Add to list
    xarray_dict[sensor] = landsat_ds

# Concatenate multiple sensors into one dataset
landsat_collectionupgrade = xr.concat(xarray_dict.values(), dim='time')
landsat_collectionupgrade = landsat_collectionupgrade.sortby('time')
landsat_collectionupgrade

## Load in current collection data (DEA Landsat Collection 2)

In [ ]:
xarray_dict = {}

for sensor in ['ls8']:

    # Load data 
    landsat_ds = dc.load(product = f'{sensor}_nbart_albers', 
                         measurements = currentcollection_bands,
                         output_crs = output_crs,
                         resolution = output_resolution,
                         resampling = output_resamp_continuous,
                         group_by = 'solar_day', 
                         **query)

    # Load PQ data 
    landsat_pq = dc.load(product = f'{sensor}_pq_albers', 
                         measurements = ['pixelquality'],
                         output_crs = output_crs,
                         resolution = output_resolution,
                         resampling = output_resamp_categorical,
                         group_by = 'solar_day', 
                         **query)                       

    # Filter to subset of Landsat observations that have matching PQ data 
    time = (landsat_ds.time - landsat_pq.time).time
    landsat_ds = landsat_ds.sel(time=time)
    landsat_pq = landsat_pq.sel(time=time)

    # Create PQ mask
    good_quality = masking.make_mask(landsat_pq.pixelquality,
                                     cloud_acca='no_cloud',
                                     cloud_shadow_acca='no_cloud_shadow',
                                     cloud_shadow_fmask='no_cloud_shadow',
                                     cloud_fmask='no_cloud',
                                     blue_saturated=False,
                                     green_saturated=False,
                                     red_saturated=False,
                                     nir_saturated=False,
                                     swir1_saturated=False,
                                     swir2_saturated=False,
                                     contiguous=True) 
    
    # Apply mask to set all PQ-affected pixels to NaN and set nodata to NaN
    landsat_ds = landsat_ds.where(good_quality)

    # Add result to dict
    xarray_dict[sensor] = landsat_ds

# Concatenate multiple sensors into one dataset
landsat_currentcollection = xr.concat(xarray_dict.values(), dim='time')
landsat_currentcollection = landsat_ds.sortby('time')
landsat_currentcollection

In [ ]:
# Round both dataset timestamps to daily because both datasets have different timestamps
landsat_collectionupgrade.coords['time'] = landsat_collectionupgrade.time.astype('datetime64[D]')
landsat_currentcollection.coords['time'] = landsat_currentcollection.time.astype('datetime64[D]')

# Rename current collection bands to match collection upgrade
band_rename_dict = {'red': 'nbart_red', 
                    'nir': 'nbart_nir' }
landsat_currentcollection.rename(band_rename_dict, inplace=True)

# Unfortunately due to the different pixel definitions for the DEA Landsat Collections 2 and 3,
# the same spatial query can result in a different number of rows and columns. Because of this,
# the only way we can directly compare the two collections is to force them to match by resampling
# one to the rows x columns of the other. This will introduce some uncertainty in the comparison.
landsat_currentcollection = landsat_currentcollection.interp_like(landsat_collectionupgrade)

## Run MaxNDVI irrigated extent algorithm on both datasets

In [ ]:
#band indices calculation
def ndvi_func(nir, red):
    return ((nir - red)/(nir + red))

def ndvi_ufunc(ds):
    return xr.apply_ufunc(
        ndvi_func, ds.nbart_nir, ds.nbart_red,
        dask='parallelized',
        output_dtypes=[float])

print("calculating NDVI")
NDVI_landsat = ndvi_ufunc(landsat_currentcollection)

#calculate per pixel summary stats
print("calculating summary stats")
NDVI_max = NDVI_landsat.groupby('x','y').max('time').rename('NDVI_max')
# NDVI_max = NDVI_max.chunk({'x':1000, 'y':1000})

transform, projection = transform_tuple(NDVI_max, (NDVI_max.x, NDVI_max.y), epsg=28355)
print("exporting MaxNDVI GTiff")
SpatialTools.array_to_geotiff(results + AOI + "_" + year + ".tif",
              NDVI_max.values, geo_transform = transform, 
              projection = projection, nodata_val=-9999.)

# ## Image Segmentation

# setup input filename
InputNDVIStats = results + AOI + "_" + year + ".tif"
KEAFile = results + AOI + '_' + year + '.kea'
SegmentedKEAFile = results + AOI + '_' + year + '_sheperdSEG.kea'
SegmentedTiffFile = results + AOI + '_' + year + '_sheperdSEG.tif'
SegmentedPolygons = results + AOI + '_' + year + '_SEGpolygons.shp'
print("calculating imageSegmentation")
imageSeg(InputNDVIStats, KEAFile, SegmentedKEAFile, SegmentedTiffFile, SegmentedPolygons, epsg='28355')

# ### Zonal Statistics & filtering

gdf = gpd.read_file(results + AOI + '_' + year + '_SEGpolygons.shp')
#calculate zonal mean of NDVI
print("Calculating zonal stats over the polygons")
gdf['mean'] = pd.DataFrame(zonal_stats(vectors=gdf['geometry'], raster=InputNDVIStats, stats='mean'))['mean']
#calculate area of polygons
gdf['area'] = gdf['geometry'].area
#filter by area and mean NDVI
highNDVI = gdf['mean'] >= threshold
smallArea = gdf['area'] <= 5500000
gdf = gdf[highNDVI & smallArea]
#export shapefile
gdf.to_file(results + AOI + "_" + year + "_Irrigated.shp")

#get the transform and projection of our gtiff
transform, projection = transform_tuple(NDVI_max, (NDVI_max.x, NDVI_max.y), epsg=28355)
#find the width and height of the xarray dataset we want to mask
width,height = NDVI_max.shape
# rasterize vector
gdf_raster = SpatialTools.rasterize_vector(results + AOI + "_" + year + "_Irrigated.shp",
                                           height, width, transform, projection, raster_path=None)
# Mask the xarray
NDVI_max_Irrigated = NDVI_max.where(gdf_raster)

#remove areas below our threshold that are at the edges of the rasterized polygons
NDVI_max_Irrigated = NDVI_max_Irrigated.where(NDVI_max_Irrigated >= threshold)

#What is the area of irrigation?
ones = np.count_nonzero(~np.isnan(NDVI_max_Irrigated.values))
area = (ones*(25*25)) / 1000000
print("Around " + AOI + " during " + str(year) + ", " + str(area) + " km2 was under irrigated cultivation")

In [ ]:
a = xr.open_rasterio(InputNDVIStats)
a.plot()

In [ ]:
landsat_collectionupgrade

In [ ]:
b = xr.open_rasterio(KEAFile)
b

## Compute difference

In [ ]:
# Subtract collection upgrade 
collection_diffs = landsat_currentcollection - landsat_collectionupgrade

# Plot differences for each band. Blue = current collection was higher than collection upgrade
collection_diffs.to_array().plot(col='variable', row='time', size=4, vmin=-200, vmax=200, cmap='RdBu', aspect=1)